In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [2]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [3]:
%matplotlib notebook

In [4]:
df = pd.read_excel('赛题1测试数据集/商标.xlsx', index_col='企业编号')
df

,商标状态,申请日期,专用权期限开始日期,专用权期限结束日期,商标使用期限时间段
企业编号,,,,,
4001,等待实质审查,2017-08-31,NaN,NaN,至
4001,等待实质审查,2017-08-31,NaN,NaN,至
4001,等待实质审查,2017-03-15,NaN,NaN,至
4001,等待实质审查,2017-08-31,NaN,NaN,至
4001,等待实质审查,2017-08-31,NaN,NaN,至
...,...,...,...,...,...
4525,商标注册申请等待受理中,2017-01-22,NaN,NaN,至
4525,等待受理通知发文,2008-12-18,NaN,2020-07-06,2010-07-07至2020-07-06
4525,初步审定公告,2018-01-22,NaN,NaN,至


In [6]:
count = pd.DataFrame(df.drop_duplicates().index.value_counts()).T
count = count.reindex(columns=list(range(4001,4526))).fillna(0)
count.index = pd.Index(['商标_count'])
count

,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
商标_count,46,39,12,25,11,1,33,60,89,31,...,6,7,139,42,3,7,46,24,4,137


In [8]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days


date_of_application = df[['申请日期']].applymap(to_day).groupby('企业编号').mean().T
date_of_application = date_of_application.reindex(columns=list(range(4001,4526)))
date_of_application

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
申请日期,-867.041002,-2810.573333,-2676.571429,-2350.882353,-2405.125,-7730.0,-4571.901961,-1616.564626,-2275.343949,-1322.509434,...,-922.666667,-2646.785714,-3419.696429,-1062.990566,-871.111111,-1178.571429,-1193.91,-2616.111111,-549.6,-2952.022556


In [9]:
def to_day_for_period(cell):
    if not isinstance(cell, str):
        return cell
    try:
        return (lambda x: (dateutil.parser.parse(x[-1]) - dateutil.parser.parse(x[0])).days)(cell.split('至'))
    except:
        pass

In [10]:
trademark_life_period = df[['商标使用期限时间段']].applymap(to_day_for_period).groupby('企业编号').mean().T
trademark_life_period = trademark_life_period.reindex(columns=list(range(4001,4526)))
trademark_life_period

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
商标使用期限时间段,3651.592593,3651.347826,3651.928571,3651.913043,3651.375,3652.0,3651.521739,3651.472973,3651.429752,3651.869565,...,3652.0,3651.0,3651.518919,3651.261905,3651.8,3651.428571,3651.741935,3651.75,3651.833333,3651.35


In [11]:
pd.concat([count, date_of_application, trademark_life_period]).to_pickle('商标.pickle-testing')